# **Scripts for Generating Outputs from Pretrained LLM Models**

---



Can do all computation on CPU. These LLMs are fairly quick.
Can either mount drive to "/content/gdrive/My\ Drive/CS247-Empathy-Mental-Health/Empathy-Mental-Health/dataset" or can just load 'ER-reddit-test.csv' locally into collab file bar on the left. I'm choosing the latter.

### Extract 'seeker_post' column from input csv file. This will contain the queries to the model.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/ER-reddit-test.csv')
input_queries = df['seeker_post']

### Output Script to write to csv file given responses array

In [ ]:
import csv
import pandas as pd

def construct_output_csv(fileName, responses):
    with open(fileName, 'w', newline='') as csvfile:
        fieldnames = ['id', 'seeker_post', 'response_post']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(input_queries)):
            writer.writerow({'id':f"{i+1}",'seeker_post':str(input_queries[i]), 'response_post':str(responses[i])})


### Obtain model responses using DalioGPT-large. Conversation based pretrained model:

In [ ]:
responses = []

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def load_tokenizer_and_model(model="microsoft/DialoGPT-large"):
  """
    Load tokenizer and model instance for some specific DialoGPT model.
  """
  # Initialize tokenizer and model
  print("Loading model...")
  tokenizer = AutoTokenizer.from_pretrained(model)
  model = AutoModelForCausalLM.from_pretrained(model)

  # Return tokenizer and model
  return tokenizer, model

def generate_response(tokenizer, model, chat_round, chat_history_ids, input_token):
  """
    Generate a response to some user input.
  """
  # Encode user input and End-of-String (EOS) token
  new_input_ids = tokenizer.encode(input_token + tokenizer.eos_token, return_tensors='pt')
  bot_input_ids = new_input_ids

  # Generate response given maximum chat length history of 1250 tokens
  chat_history_ids = model.generate(bot_input_ids, max_length=1250, pad_token_id=tokenizer.eos_token_id)

  # print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

  responses.append(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))
  # Return the chat history ids
  return chat_history_ids

def obtain_responses_to_queries():
  tokenizer, model = load_tokenizer_and_model()

  for chat_round in range(len(input_queries)):
      if chat_round % 20 == 0: print(chat_round)
      generate_response(tokenizer, model, chat_round, None, str(input_queries[chat_round]))

obtain_responses_to_queries()


In [ ]:
print(responses)

["I know what you're doing. I know what you're doing. I know what you're doing. I know what you're doing. I know what you're doing.", 'I know that feel.', "I feel like I'm being criticized and mocked even when aloneothing.", 'I feel you. I have a lot of anxiety and depression.', 'hugs', 'I know that feel.', "You can't get financial assistance if you're under 18.", 'I hope you feel better soon!', "I'm sorry to hear that. I hope you find a good home.", "I'm not sure if you're being serious or not.", 'I have BPD and I have ADHD. I have been diagnosed with BPD. I have been diagnosed with Bipolar. I have been diagnosed with BPD.', "I think it's over.", '', "I'm not sure what you mean by depression.", 'I know that feel.', "I know, I know. I'm sorry.", 'I know that feel.', "I'm not sure why you're being downvoted.", "I know, I'm sorry.", "I'm not sure.", 'I know, right? I wish I had never existed.', "Because it's a death sentence.", "I'm in a bad place too. I'm in a bad place.", "I'm not sure

### Construct output csv file for dalio gpt.

In [ ]:
construct_output_csv('dalio_gpt.csv', responses)

### Obtain model responses using Roberta Base. Next-token based prediction model. Should expect very bad output.

In [ ]:
from transformers import AutoTokenizer, RobertaForCausalLM, AutoConfig
import torch

responses = []

def load_tokenizer_and_model(model="FacebookAI/roberta-base"):
    """
      Load tokenizer and model instance for some specific DialoGPT model.
    """
    # Initialize tokenizer and model
    print("Loading model...")
    tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
    model = RobertaForCausalLM.from_pretrained("FacebookAI/roberta-base")

    # Return tokenizer and model
    return tokenizer, model

def respond_to_queries(tokenizer, model):
    for chat_entry in range(len(input_queries)):
        input_text = input_queries[chat_entry]
        inputs = tokenizer(input_text, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        prediction_logits = outputs.logits
        predicted_text = tokenizer.decode(torch.argmax(prediction_logits, dim=-1)[0], skip_special_tokens=True)
        responses.append(predicted_text)

tokenizer, model = load_tokenizer_and_model()
respond_to_queries(tokenizer, model)

Loading model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
print(responses)

["Help. Help me I dunno what I'm doing anymore", "I'm done saying I love you to her because I don't want to hear a lie back to me. I feel so ugly and unwanted and unloved", "Always feel like I'm being criticized and mocked even when alone.... Does anyone else experience this?", "My diet becomes fucked when i get depressed.. I can't control myself in the grocery store when I'm feeling down. Chips, cookies, soda, cake, you name It Anyone else a stress eater?", "I hate not knowing why. I was diagnosed with depression and anxiety when I was 14. That's almost 4 years ago and I still don't know why I am like this. I have more than enough, yet there are moments where I feel like there's no hope nor a bright future waiting for me. I feel alone and abandoned even though I have friends who know that I am depressed and am like this sometimes. I get so frustrated when I can't find out why I just can't be normal. Everything was alright about 2 hours ago, but it just changes so quickly and hard that

### Construct csv output file for roberta base

In [ ]:
construct_output_csv('roberta_base.csv', responses)

### Obtain Model Responses using GPT-2.

In [ ]:
!pip install trl
!git clone https://github.com/huggingface/trl.git
!cd trl/
!pip install -e .

In [ ]:
from trl.core import respond_to_batch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

responses = []

def load_tokenizer_and_model(model="gpt2"):
    """
      Load tokenizer and model instance for some specific DialoGPT model.
    """
    # Initialize tokenizer and model
    print("Loading model...")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    # Return tokenizer and model
    return tokenizer, model

def respond_to_queries(tokenizer, model):
    for chat_entry in range(len(input_queries)):
        user_input = input_queries[chat_entry]
        query_tensor = tokenizer.encode(user_input, return_tensors="pt")
        response_tensor = respond_to_batch(model, query_tensor)
        response = tokenizer.decode(response_tensor[0], skip_special_tokens=True)
        responses.append(response)

tokenizer, model = load_tokenizer_and_model()
respond_to_queries(tokenizer, model)

Loading model...


### Create output csv file for GPT2 responses

In [ ]:
construct_output_csv('gpt2.csv', responses)

# RL Playground

The goal is to achieve the following pipeline:
1. Input tensor list of queries
2. Output tensor list of responses
3. Convert to csv for input to evaluator
4. Pass through evaluator to obtain output csv with evaluator scores.
5. Parse output csv to extract scores.
6. Utilize scores to compute reward
7. Push input, output, and reward tensors to trl to perform train step.

Edit: I've achieved this pipeline with individual queries, goal is now to do this in batches so that it's at least reasonably fast.

Edit: Batches can't be done :( That's not how this model works, and a slow evaluator is kind of a dead end. I will train as much as I can, and we'll see if we can get 10,000 data points in, but will definitely be difficult.

In [ ]:
#Imports and Definitions

import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

# get models
model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
model_ref = create_reference_model(model)

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# initialize trainer
ppo_config = {"mini_batch_size": 1, "batch_size": 1}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer)

Example of steps 1,2, and 7 in the pipeline. We use a constant value for reward function.

In [ ]:
#example of one single train iteration
query_txt = input_queries[0]
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").cuda()

# get model response
response_tensor  = respond_to_batch(model, query_tensor)

# reward -- just a constant for this example
reward = [torch.tensor(1.0)]

# train model for one step with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [ ]:
import os
import shutil

try:

    from google.colab import drive
    drive.mount('/content/gdrive')

    DRIVE_PATH = '/content/gdrive/My\ Drive/CS247-Empathy-Mental-Health'
    DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
    if not os.path.exists(DRIVE_PYTHON_PATH):
      %mkdir $DRIVE_PATH

    ## the space in `My Drive` causes some issues,
    ## make a symlink to avoid this
    # Solved -> symlink for convenience
    SYM_PATH = '/content/CS247-Empathy-Mental-Health'
    if not os.path.exists(SYM_PATH):
      !ln -s $DRIVE_PATH $SYM_PATH

    running_in_colab = True

    # We already mounted in our google drive.
    # Enter the foler where you put files in:
    %cd '/content/CS247-Empathy-Mental-Health'

    # What files are there:
    !ls


except ModuleNotFoundError:
    running_in_colab = False
    print(
        "I guess you are running locally. If you get this message in Colab, check the files."
    )

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1qwurxfG3wTYT_VY1AQ0AaMBt4LPZf23w/CS247-Empathy-Mental-Health
 best_emotion.pt	   NaiveBaselineModel.ipynb	   rlhf_q_2
 checkpoint_other_131.pt   output			   rlhf_q_3
 checkpoint_other_79.pt    PretrainedModelQuerying.ipynb   rlhf_question_0_100
 Empathy-Mental-Health	   rlhf_default_0_100		   rlhf_question_0_200
 EmpDialogue_RecEC	   rlhf_default_0_200		   rlhf_question_0_300
 ER-reddit-test.csv	   rlhf_default_logs.json	   rlhf_question_logs.json
'Generative Model.ipynb'   rlhf_length_0		   rlhf_therapist_length_0_100
 glove.6B.100d.txt	   rlhf_length_0_100		   rlhf_therapist_length_0_200
 glove.6B.200d.txt	   rlhf_length_0_200		   rlhf_therapist_length_0_300
 glove.6B.300d.txt	   rlhf_length_0_300		   rlhf_therapist_length_logs.json
 glove.6B.50d.txt	   rlhf_length_logs.json	  'RL Training.ipynb'
 hard-gate-test.gdoc	  'RLHF on SFT'			   roberta-large.tsv
 hard-gate-test.txt	   rlhf_q_1			   SFT_GPT2


In [ ]:
%cd Empathy-Mental-Health/
!ls

[Errno 2] No such file or directory: 'Empathy-Mental-Health/'
/content/gdrive/.shortcut-targets-by-id/1eUJZcBYmEsh0qtMhIO6uDeGrZPPBmQio/Empathy-Mental-Health
'Althoff academic license.docx'		  dataset   README.md	       src
'Althoff attribution only license.docx'   output    requirements.txt   train.sh


This helps setup steps 3-6 in the pipeline. All functions for converting/parsing csv, extracting reward value are here.

In [ ]:
def convert_csv_for_evaluator(fileName, query, response):
    with open(fileName, 'w', newline='') as csvfile:
        fieldnames = ['id', 'seeker_post', 'response_post']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow({'id':'1','seeker_post':str(query), 'response_post':str(response)})

def calculate_reward_function_using_evaluator():
    !python3 src/test.py \
	--input_path /content/evaluator_input.csv \
	--output_path /content/evaluator_output.csv \
	--ER_model_path output/reddit_ER.pth \
	--IP_model_path output/reddit_IP.pth \
	--EX_model_path output/reddit_EX.pth
    return

#Algorithm to compute reward. Currently just summing up all 3 labels and normalizing to this scale:
#6->3, 5->2, 4->1, 3->0, 2->-1, 1->-2, 0->-3
def compute_reward_single_value(ER_label, IP_label, EX_label):
    return [torch.tensor((ER_label + IP_label + EX_label) - 3, dtype=torch.float32)]


def extract_reward_from_output_csv_from_evaluator_single_value():
    input_df = pd.read_csv('/content/evaluator_output.csv', header=0)
    ER_label = int(input_df.ER_label.astype(str).tolist()[0])
    IP_label = int(input_df.IP_label.astype(str).tolist()[0])
    EX_label = int(input_df.EX_label.astype(str).tolist()[0])
    return compute_reward_single_value(ER_label, IP_label, EX_label)


Steps 1-7 for a single input query:

In [ ]:
chat_entry = input_queries[0]
query_tensor = tokenizer.encode(chat_entry, return_tensors="pt").cuda()

# get model response
response_tensor  = respond_to_batch(model, query_tensor)
response = tokenizer.decode(response_tensor[0], skip_special_tokens=True)

# use query, response to obtain evaluator reward score
convert_csv_for_evaluator('/content/evaluator_input.csv', chat_entry, response)
calculate_reward_function_using_evaluator()
reward = extract_reward_from_output_csv_from_evaluator_single_value()
# # train step based on query, resonse, reward
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

print("query: ", chat_entry)
print("response: ", response)
print("reward", reward)

The following code will train using ppo on all of the queries in the list "input_queries". However, it does so only one query at a time, so we have to establish the full pipeline for each query. Thus, very very slow. About 1 min per query.

In [ ]:
for chat_entry in input_queries:
    print("query: ", chat_entry)
    query_tensor = tokenizer.encode(chat_entry, return_tensors="pt").cuda()

    # get model response
    response_tensor  = respond_to_batch(model, query_tensor)
    response = tokenizer.decode(response_tensor[0], skip_special_tokens=True)
    print("response: ", response)

    # use query, response to obtain evaluator reward score
    convert_csv_for_evaluator('/content/evaluator_input.csv', chat_entry, response)
    calculate_reward_function_using_evaluator()
    print("done forwarding through evaluator")
    reward = extract_reward_from_output_csv_from_evaluator_single_value()
    print("reward", reward)
    # # train step based on query, resonse, reward
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

query:  Help. Help me. I dunno what I'm doing anymore
response:  . Just get this boy kicked out of my god-shitclaw. Photo Archives Prev Next Field Administrator
2024-03-08 23:02:08.896714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 23:02:08.896778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 23:02:08.903163: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 23:02:10.863083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Truncation was not explicitly activated but `max_length` is provided a specific 

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1279: UserWarning: KL divergence is starting to become negative: -1.46 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


query:  All the people who will be kissed on New year's eve. I'll be alone like usual I'll never get someone to kiss me It's fine No one understands how hopelessly, alone, and angrily some people live. It would scare some attractive people that haven't ever lived like that.
response:   It's beautiful. And it's means that you can, me you dare, I can multiply desires
2024-03-08 23:18:41.517798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 23:18:41.517856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 23:18:41.523580: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been register